In [77]:
import requests
from bs4 import BeautifulSoup
import time
import urllib.request #
from selenium.webdriver import Chrome
import re     
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import datetime as dt
import pandas as pd

In [80]:
os.chdir('C:\\Users\\shic\\Desktop\\Project')

In [81]:
delay=3
browser = Chrome()
browser.implicitly_wait(delay)

In [82]:
df = pd.DataFrame({'name':[],
                'title':[],
                'date':[],
                'video_url':[],
                'view_num':[],
                'likes_num':[],
                'unlikes_num':[],
                'comments':[]})
df

,name,title,date,video_url,view_num,likes_num,unlikes_num,comments


In [84]:
start_url  = 'https://www.youtube.com/c/%EC%82%BC%ED%94%84%EB%A1%9Ctv/videos'
browser.get(start_url)  
body = browser.find_element_by_tag_name('body')#스크롤하기 위해 소스 추출

num_of_pagedowns = 200
browser.maximize_window() #전체화면이 아니라 최대화하는 방법
#10번 밑으로 내리는 것
while num_of_pagedowns:
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(0.1)
    num_of_pagedowns -= 1
html0 = browser.page_source
html = BeautifulSoup(html0,'html.parser')
video_list = html.find_all('a',{'class':'yt-simple-endpoint style-scope ytd-grid-video-renderer'})

In [85]:
len(video_list)

3600

In [90]:
base_url = 'http://www.youtube.com'
spro_url = []
for i in range(len(video_list)):
    url = base_url+video_list[i]['href']
    spro_url.append(url)

In [96]:
len(spro_url)

3600

In [112]:
count=0
for i in range(451,len(video_list)):
    name = video_list[i]['title']
    start_url = spro_url[i]
    browser.get(start_url)
    body = browser.find_element_by_tag_name('body')
    num_of_pagedowns = 10
    while num_of_pagedowns:
        body.send_keys(Keys.PAGE_DOWN)
        time.sleep(1)
        num_of_pagedowns -= 1
    body = browser.find_element_by_tag_name('body')
    
    time.sleep(1)#브라우저 로딩시간기다려야함
    browser.maximize_window()#전체화면이 아니라 최대화하는 방법
    num_of_pagedowns = 2 #page_down은 2로 놔도 괜찮을 듯 최대화 했으므로

    html0 = browser.page_source
    html = BeautifulSoup(html0,'html.parser')
    title = html.find_all('yt-formatted-string',{'class':'style-scope ytd-video-primary-info-renderer'})[0].text
    date = html.find_all('yt-formatted-string',{'class':'style-scope ytd-video-primary-info-renderer'})[1].text
    view_num = html.find_all('span',{'class':'view-count style-scope ytd-video-view-count-renderer'})[0].text
    likes_num =html.find_all('yt-formatted-string',{'id':'text','class':'style-scope ytd-toggle-button-renderer style-text'})[0]['aria-label']
    unlikes_num = html.find_all('yt-formatted-string',{'id':'text','class':'style-scope ytd-toggle-button-renderer style-text'})[1]['aria-label']

    html_s0 = browser.page_source
    html = BeautifulSoup(html_s0,'html.parser')

    # 댓글 60개
    comment_list=[]
    com = html.find_all('yt-formatted-string',{'class':'style-scope ytd-comment-renderer','id':'content-text'})
    for i,item in enumerate(com):
        comment_list.append(item.text)
    comment_all = "|".join(comment_list)
    
    record  = pd.Series([name,title, date,start_url,view_num,likes_num,unlikes_num,comment_all],index = ['name','title','date','video_url','view_num','likes_num','unlikes_num','comments'])
    df = df.append(record, ignore_index = True)

    count+=1
    if count%10==False:
        print(count)
        print(record)

10
name           [심층 인터뷰] 자회사들의 상장이 지주사의 가치를 떨어트릴까? f.머니투데이방송 권...
title          [심층 인터뷰] 자회사들의 상장이 지주사의 가치를 떨어트릴까? f.머니투데이방송 권...
date                                                2021. 5. 31.
video_url             http://www.youtube.com/watch?v=DDdraQ1Qh0w
view_num                                             조회수 28,726회
likes_num                                               좋아요 894개
unlikes_num                                               싫어요 6개
comments       삼프로TV 주식대학 '봄학기'\r\n👉 염블리의 친절한 주식 기초 교육 / 미주미 ...
dtype: object
20
name           [글로벌 이슈체크] 중국엔 백주만 있다? 성장세 좋은 맥주와 소비재 산업  f. 미...
title          [글로벌 이슈체크] 중국엔 백주만 있다? 성장세 좋은 맥주와 소비재 산업  f. 미...
date                                                2021. 5. 28.
video_url             http://www.youtube.com/watch?v=rWQeaVZX9F8
view_num                                             조회수 14,190회
likes_num                                               좋아요 223개
unlikes_num                                              싫어요 13개
comme

TimeoutException: Message: timeout: Timed out receiving message from renderer: 11.958
  (Session info: chrome=91.0.4472.124)


In [98]:
df # video_url 삭제하기!

,name,title,date,video_url,view_num,likes_num,unlikes_num,comments
0,"[심층 인터뷰] 조선업의 업황 자체는 호황, 하지만 과거와 다르다? f.메리츠증권 ...","[심층 인터뷰] 조선업의 업황 자체는 호황, 하지만 과거와 다르다? f.메리츠증권 ...",2021. 7. 16.,http://www.youtube.com/watch?v=qmFDs8cyNtE,조회수 568회,좋아요 69개,싫어요 2개,"삼프로TV 주식대학 '여름학기' \r\n \n👉 [삼프로TV 주식대학] 박병창의 ""..."
1,"[뉴스3] 뉴욕증시, 파월 완화적 기조 유지에 혼조… 다우 0.15% 상승 마감","[뉴스3] 뉴욕증시, 파월 완화적 기조 유지에 혼조… 다우 0.15% 상승 마감",2021. 7. 16.,http://www.youtube.com/watch?v=qmFDs8cyNtE,조회수 808회,좋아요 99개,싫어요 2개,"삼프로TV 주식대학 '여름학기' \r\n \n👉 [삼프로TV 주식대학] 박병창의 ""..."
2,"하반기 조선업 투자 전략, 베스트 애널리스트의 픽은?_오늘아침 라이브_김현, 염승환","하반기 조선업 투자 전략, 베스트 애널리스트의 픽은?_오늘아침 라이브_김현, 염승환",이전 실시간 스트리밍: 3시간 전,http://www.youtube.com/watch?v=qmFDs8cyNtE,"조회수 200,794회","좋아요 7,137개",싫어요 114개,"삼프로TV 주식대학 '여름학기' \r\n \n👉 [삼프로TV 주식대학] 박병창의 ""..."
3,[글로벌 개장시황] 도망쳐! 갑자기 나타나는 밈주식의 급락세 f. 미국주식에 미치다...,[글로벌 개장시황] 도망쳐! 갑자기 나타나는 밈주식의 급락세 f. 미국주식에 미치다...,2021. 7. 16.,http://www.youtube.com/watch?v=qmFDs8cyNtE,"조회수 29,690회",좋아요 721개,싫어요 19개,"삼프로TV 주식대학 '여름학기' \n\n👉 [삼프로TV 주식대학] 박병창의 ""매매의..."
4,[글로벌 이슈체크] 상반기와 확연히 다른 하반기 중국 투자 접근법 f. KB증권 박...,[글로벌 이슈체크] 상반기와 확연히 다른 하반기 중국 투자 접근법 f. KB증권 박...,2021. 7. 16.,http://www.youtube.com/watch?v=qmFDs8cyNtE,"조회수 7,786회",좋아요 189개,싫어요 2개,"삼프로TV 주식대학 '여름학기' \r\n \n👉 [삼프로TV 주식대학] 박병창의 ""..."
5,"[글로벌마켓 브리핑] 자율주행은 데이터 산업, 새로운 승자가 나타난다! f. 미래에...","[글로벌마켓 브리핑] 자율주행은 데이터 산업, 새로운 승자가 나타난다! f. 미래에...",2021. 7. 16.,http://www.youtube.com/watch?v=qmFDs8cyNtE,"조회수 11,398회",좋아요 358개,싫어요 11개,"삼프로TV 주식대학 '여름학기' \r\n \n👉 [삼프로TV 주식대학] 박병창의 ""..."
6,[글로벌라이브] 상반기와 확연히 다른 하반기 중국 투자 접근법 f. KB증권 박수...,[글로벌라이브] 상반기와 확연히 다른 하반기 중국 투자 접근법 f. KB증권 박수...,이전 실시간 스트리밍: 14시간 전,http://www.youtube.com/watch?v=qmFDs8cyNtE,"조회수 96,777회","좋아요 2,175개",싫어요 51개,"삼프로TV 주식대학 '여름학기' \r\n \n👉 [삼프로TV 주식대학] 박병창의 ""..."
7,"[일당백 인터뷰] 전기차 전용 보험, 어디까지 알아보셨어요? f.한겨레신문 박종오 기자","[일당백 인터뷰] 전기차 전용 보험, 어디까지 알아보셨어요? f.한겨레신문 박종오 기자",2021. 7. 15.,http://www.youtube.com/watch?v=qmFDs8cyNtE,"조회수 6,011회",좋아요 143개,싫어요 8개,"삼프로TV 주식대학 '여름학기' \r\n \n👉 [삼프로TV 주식대학] 박병창의 ""..."
8,[주인장 백브리핑] 1년 3개월 만에 나온 금통위 소수의견의 의미,[주인장 백브리핑] 1년 3개월 만에 나온 금통위 소수의견의 의미,2021. 7. 15.,http://www.youtube.com/watch?v=qmFDs8cyNtE,"조회수 6,616회",좋아요 182개,싫어요 2개,"삼프로TV 주식대학 '여름학기' \r\n \n👉 [삼프로TV 주식대학] 박병창의 ""..."
9,"[퇴근길 라이브&백브리핑 라이브] 유동성 흡수 미루는 美연준, 증시는 계속 오르나?...","[퇴근길 라이브&백브리핑 라이브] 유동성 흡수 미루는 美연준, 증시는 계속 오르나?...",이전 실시간 스트리밍: 17시간 전,http://www.youtube.com/watch?v=qmFDs8cyNtE,"조회수 181,790회","좋아요 4,818개",싫어요 83개,"삼프로TV 주식대학 '여름학기' \r\n \n👉 [삼프로TV 주식대학] 박병창의 ""..."


In [103]:
df.shape

(356, 8)

In [68]:
df.iloc[7].comments

'삼프로TV 주식대학 \'여름학기\' \r\n \n👉 [삼프로TV 주식대학] 박병창의 "매매의 기술" + 염승환의 "친절한 주식기초교육"\r\n🏫 강의 신청 : https://bit.ly/3r1osDg\r\n🏫 Bonus-Edition "쏙쏙 투자문답" 질문하기 : https://bit.ly/3rbHG9j\r\n \n👉 [삼프로TV x 메리츠증권] 하반기 투자전략과 업종별 시장전망\r\n🏫 강의 신청 : https://bit.ly/3xn1OXM\r\n \n삼프로TV 주식대학 \'봄학기\'\r\n👉 염블리의 친절한 주식 기초 교육 / 미주미 장우석 본부장, 이항영 대표의 미국주식 완전정복\r\n🏫 강의 신청 : https://www.page-2.kr/concert/2021_jurinee\r\n \n삼프로TV "시장전망과 주도종목 : https://www.page-2.kr/purchase/course\r\n \n👉 삼프로 TV 주식대학 "업종 기초 교육"\r\n🏫 강의 신청 : https://bit.ly/39BG9lo\r\n📖 교재 구매 : https://bit.ly/2Ol36BC \n \n*구매하시는 교재는 강의 신청 시 첨부되는 PDF 교재와 동일한 내용입니다.\r\n \n★ 광고문의 : ad@ebroadcasting.co.kr \n★ 광고문의 : 010-2090-6748\r\n★ 강의 및 page2 서비스 문의 : 02-2115-2324 , 010-3898-2324\r\n \n#경제의신과함께 #삼프로TV #신과함께 \n00:00 오프닝\r\n04:10 [증시 셔터맨] 발톱 드러낸 금통위... 금리 동결, 물가 상승 f. 한국투자증권 박제영 차장\r\n24:12 [심층 인터뷰] 예고된 금리 인상, 투자자의 탄력적 대응은 어떻게? f. 하나금융투자 이미선 연구위원\r\n1:12:46 [주인장 백브리핑] 1년 3개월 만에 나온 금통위 소수의견의 의미\r\n1:41:00 [일당백 인터뷰] 전기차 전용 보험, 어디까지 알아보셨어요? f.한겨레신문 박종오 기자|이센터장

In [104]:
import os
os.chdir('C:\\Users\\shic\\Desktop\\Project\\1_Data')

In [110]:
df.to_csv('youtube_eco_3pro_6.csv',index=False)

In [111]:
df_ = pd.read_csv('youtube_eco_3pro_6.csv')
df_.shape

(356, 8)

In [73]:
df_

,name,title,date,video_url,view_num,likes_num,unlikes_num,comments
0,[글로벌 개장시황] 도망쳐! 갑자기 나타나는 밈주식의 급락세 f. 미국주식에 미치다...,[글로벌 개장시황] 도망쳐! 갑자기 나타나는 밈주식의 급락세 f. 미국주식에 미치다...,2021. 7. 16.,http://www.youtube.com/watch?v=v5QDWVaULyo,"조회수 19,414회",좋아요 642개,싫어요 18개,"삼프로TV 주식대학 '여름학기' \n\n👉 [삼프로TV 주식대학] 박병창의 ""매매의..."
1,[글로벌 개장시황] 도망쳐! 갑자기 나타나는 밈주식의 급락세 f. 미국주식에 미치다...,[글로벌 개장시황] 도망쳐! 갑자기 나타나는 밈주식의 급락세 f. 미국주식에 미치다...,2021. 7. 16.,http://www.youtube.com/watch?v=v5QDWVaULyo,"조회수 19,414회",좋아요 642개,싫어요 18개,"삼프로TV 주식대학 '여름학기' \n\n👉 [삼프로TV 주식대학] 박병창의 ""매매의..."
2,[글로벌 이슈체크] 상반기와 확연히 다른 하반기 중국 투자 접근법 f. KB증권 박...,[글로벌 이슈체크] 상반기와 확연히 다른 하반기 중국 투자 접근법 f. KB증권 박...,2021. 7. 16.,http://www.youtube.com/watch?v=v5QDWVaULyo,"조회수 4,969회",좋아요 160개,싫어요 2개,"삼프로TV 주식대학 '여름학기' \r\n \n👉 [삼프로TV 주식대학] 박병창의 ""..."
3,"[글로벌마켓 브리핑] 자율주행은 데이터 산업, 새로운 승자가 나타난다! f. 미래에...","[글로벌마켓 브리핑] 자율주행은 데이터 산업, 새로운 승자가 나타난다! f. 미래에...",2021. 7. 16.,http://www.youtube.com/watch?v=v5QDWVaULyo,"조회수 7,929회",좋아요 317개,싫어요 10개,"삼프로TV 주식대학 '여름학기' \r\n \n👉 [삼프로TV 주식대학] 박병창의 ""..."
4,[글로벌라이브] 상반기와 확연히 다른 하반기 중국 투자 접근법 f. KB증권 박수...,[글로벌라이브] 상반기와 확연히 다른 하반기 중국 투자 접근법 f. KB증권 박수...,이전 실시간 스트리밍: 11시간 전,http://www.youtube.com/watch?v=v5QDWVaULyo,"조회수 92,713회","좋아요 2,136개",싫어요 47개,"삼프로TV 주식대학 '여름학기' \r\n \n👉 [삼프로TV 주식대학] 박병창의 ""..."
...,...,...,...,...,...,...,...,...
61,"[고민 상담] 장기투자, 트레이딩 어떻게 섞어서 해야할까요 (박세익 전무 & 조형우...","[고민 상담] 장기투자, 트레이딩 어떻게 섞어서 해야할까요 (박세익 전무 & 조형우...",2021. 7. 10.,http://www.youtube.com/watch?v=v5QDWVaULyo,"조회수 182,428회","좋아요 5,267개",싫어요 116개,사연 신청은 counsel@ebroadcasting.co.kr 로 연락처와 함께 보...
62,[글로벌 개장시황] 미국 36개 주에서 고소당한 구글의 앞날은? f. 미국주식에 미...,[글로벌 개장시황] 미국 36개 주에서 고소당한 구글의 앞날은? f. 미국주식에 미...,2021. 7. 9.,http://www.youtube.com/watch?v=v5QDWVaULyo,"조회수 40,383회","좋아요 1,001개",싫어요 46개,"삼프로TV 주식대학 '여름학기' \r\n \n👉 [삼프로TV 주식대학] 박병창의 ""..."
63,[글로벌 이슈체크] 금리 인상 가시화? 하반기 투자 중점 사항은? _f. 미래에셋증...,[글로벌 이슈체크] 금리 인상 가시화? 하반기 투자 중점 사항은? _f. 미래에셋증...,2021. 7. 9.,http://www.youtube.com/watch?v=v5QDWVaULyo,"조회수 16,840회",좋아요 304개,싫어요 14개,"삼프로TV 주식대학 '여름학기' \r\n \n👉 [삼프로TV 주식대학] 박병창의 ""..."
64,"[글로벌마켓 브리핑] 완성차 세계 4위, 전기차 올인하는 '스텔란티스' f. 미래에...","[글로벌마켓 브리핑] 완성차 세계 4위, 전기차 올인하는 '스텔란티스' f. 미래에...",2021. 7. 9.,http://www.youtube.com/watch?v=v5QDWVaULyo,"조회수 16,940회",좋아요 389개,싫어요 14개,"삼프로TV 주식대학 '여름학기' \r\n \n👉 [삼프로TV 주식대학] 박병창의 ""..."
